# Visualizing hyperspectral signatures of agricultural crops 

Author: Ivan Lizarazo

Date: 23.10.2024

## Goal

This notebook aims at illustrating how to interactively explore hyperspectral signatures of vegetation using several Python libraries. It may be useful in a variety of remote sensing of vegetation applications. 


![](./GHISA/GHISA_quasi.png)

## A short refresher on remote sensing of vegetation

High spectral resolution measurements of leaves and plant canopies enable the indirect, non-contact measurement of key structural and chemical absorption features that are associated with the physiological and biochemical properties of plants.

![](./GHISA/signatures.png)

Together, leaf optical properties and canopy architecture regulate the remote sensing signatures observed in remote sensing data. In addition, changes in leaf internal biochemistry or structure (i.e., functional traits) as a result of biotic or abiotic factors can change these signatures over space and time. For example, a prolonged drought can cause changes in leaf internal water content and potentially a redistribution of internal pigmentation. Next figure illustrates the changes in leaf an canopy spectra over the course of a low, moderate, and high drought event.

![](./GHISA/signatures2.png)

For more information on optical properties of leaves and canopies see the Ustin & Jacquemoud's chapter on the book [Remote Sensing of Plant
Biodiversity](https://link.springer.com/chapter/10.1007/978-3-030-33157-3_14) (Cavender-Bares et. al., 2020)

## Setup

First, let’s import the necessary dependencies and define some variables:

In [3]:
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn

import matplotlib.pyplot as plt
import seaborn as sns

colors = ["#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA"]
COLOR1 = colors[0]
COLOR2 = colors[1]
COLOR3 = colors[2]
COLOR4 = colors[3]
COLOR5 = colors[4]
COLOR6 = colors[5]
COLOR7 = colors[6]

Next, we’ll import the Panel JavaScript dependencies using pn.extension(...). For a visually appealing and responsive user experience, we’ll set the design to "material" and the sizing_mode to stretch_width:

In [4]:
pn.extension(design="material", sizing_mode="stretch_width")

## GHISA dataset

![](./GHISA/GHISA_dataset.png)

The Global Hyperspectral Imaging Spectral-library of Agricultural crops (GHISA) is a comprehensive compilation, collation, harmonization, and standardization of hyperspectral signatures of agricultural crops of the world. This hyperspectral library of agricultural crops is developed for all major world crops and was collected by United States Geological Survey (USGS) and partnering volunteer agencies from around the world. Crops include wheat, rice, barley, corn, soybeans, cotton, sugarcane, potatoes, chickpeas, lentils, and pigeon peas, which together occupy about 65% of all global cropland areas. The GHISA spectral libraries were collected and collated using spaceborne, airborne (e.g., aircraft and drones), and ground based hyperspectral imaging spectroscopy.

The GHISA for the Conterminous United States (GHISACONUS) Version 1 product provides dominant crop data in different growth stages for various agroecological zones (AEZs) of the United States (Thenkabail, P. & Aneece, I., ). The GHISA hyperspectral library of the five major agricultural crops (e.g., winter wheat, rice, corn, soybeans, and cotton) for CONUS was developed using Earth Observing-1 (EO-1) Hyperion hyperspectral data acquired from 2008 through 2015 from different AEZs of CONUS using the United States Department of Agriculture (USDA) Cropland Data Layer (CDL) as reference data.

GHISACONUS is comprised of seven AEZs throughout the United States covering the major agricultural crops in six different growth stages: emergence/very early vegetative (Emerge VEarly), early and mid vegetative (Early Mid), late vegetative (Late), critical, maturing/senescence (Mature Senesc), and harvest. The crop growth stage data were derived using crop calendars generated by the Center for Sustainability and the Global Environment (SAGE), University of Wisconsin-Madison.

The GHISACONUS v001 data set can be downloaded from the NASA's [Land Processes Distributed Active Archive Center (LP DAAC)](https://lpdaac.usgs.gov/products/ghisaconusv001/). It is provided in a CSV file which besides the spectral library includes image information, geographic coordinates, corresponding agroecological zone, crop type labels, and crop growth stage labels for the United States.

## Load the data

Now, let’s load the GHISA for CONUS hyperspectral dataset.

In [5]:
CSV_FILE = (
    "./crops/data/GHISACONUS_2008_001_speclib.csv")

In [6]:
data = pd.read_csv(CSV_FILE)

A bit of data exploration follows:

In [7]:
data.shape

(6988, 209)

In [8]:
data.tail()

,UniqueID,Country,XAEZ,Image,Month,Year,jd,long,lat,Crop,...,X2305,X2315,X2325,X2335,X2345,X2355,X2365,X2375,X2385,X2395
6983,13424,USA,10,EO1H0230322015238110K1_SG1_01,8,2015,238,-88.456936,39.522714,soybean,...,6.763665,6.813507,7.034937,7.364454,7.585006,NaN,NaN,NaN,NaN,NaN
6984,13427,USA,10,EO1H0230322015238110K1_SG1_01,8,2015,238,-88.467407,39.514106,soybean,...,6.690982,6.483347,5.838236,5.723123,5.664319,NaN,NaN,NaN,NaN,NaN
6985,13431,USA,10,EO1H0230322015238110K1_SG1_01,8,2015,238,-88.419342,39.508411,soybean,...,7.664325,6.842533,6.197422,5.654009,5.611239,NaN,NaN,NaN,NaN,NaN
6986,13432,USA,10,EO1H0230322015238110K1_SG1_01,8,2015,238,-88.425007,39.518874,soybean,...,6.831199,6.964757,6.912464,6.050916,5.549993,NaN,NaN,NaN,NaN,NaN
6987,13433,USA,10,EO1H0230322015238110K1_SG1_01,8,2015,238,-88.447838,39.513841,soybean,...,6.767493,6.209349,5.916585,5.906694,6.501351,NaN,NaN,NaN,NaN,NaN


In [9]:
data.Crop.unique()

array(['corn', 'rice', 'cotton', 'soybean', 'winter_wheat'], dtype=object)

In [10]:
##### Analyzing GHISA data #######
data.describe()

,UniqueID,XAEZ,Month,Year,jd,long,lat,X427,X437,X447,...,X2305,X2315,X2325,X2335,X2345,X2355,X2365,X2375,X2385,X2395
count,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000,0.0,6988.000000,6988.000000,...,6988.000000,6988.000000,6988.000000,6984.000000,6969.000000,0.0,0.0,0.0,0.0,0.0
mean,8349.214940,8.076417,7.766171,2012.596308,219.470950,-94.935002,39.079023,NaN,18.832709,17.073541,...,14.100354,14.047598,13.143593,12.110412,11.286820,NaN,NaN,NaN,NaN,NaN
std,3355.883373,1.967504,1.406497,1.660552,43.373447,6.655487,4.064115,NaN,2.594604,2.464195,...,8.992708,9.092754,8.599614,8.024795,7.614223,NaN,NaN,NaN,NaN,NaN
min,1466.000000,2.000000,5.000000,2008.000000,123.000000,-121.757011,31.666002,NaN,13.261094,12.077974,...,2.413298,1.555780,1.639838,1.162671,0.843631,NaN,NaN,NaN,NaN,NaN
25%,5439.750000,6.000000,7.000000,2012.000000,184.000000,-97.179724,36.769552,NaN,16.901768,15.268908,...,6.619031,6.440540,5.913871,5.373689,4.962867,NaN,NaN,NaN,NaN,NaN
50%,8670.000000,9.000000,8.000000,2013.000000,225.000000,-96.751871,39.738262,NaN,18.823415,17.040686,...,10.923218,10.925152,10.243445,9.505040,8.816483,NaN,NaN,NaN,NaN,NaN
75%,11231.500000,10.000000,9.000000,2014.000000,253.000000,-88.366985,40.449560,NaN,20.398661,18.551474,...,21.205351,21.231158,19.984068,18.306030,17.037112,NaN,NaN,NaN,NaN,NaN
max,13433.000000,10.000000,10.000000,2015.000000,302.000000,-88.085632,45.724274,NaN,31.873265,29.102861,...,51.057459,52.033016,49.394378,47.733333,46.932524,NaN,NaN,NaN,NaN,NaN



Note that, in addition to 242 hyperspectral bands (i.e. X427 to X2395), each of 10 nm bandwidth in the 400-2500 nm range, the GHISA dataset includes:

- Unique ID : identifyig a unique crop field
- XAEZ : identifying the Agrogeological Zone (all in USA)
- Month : month in which the image was acquired
- Year : year in which the image was acquired
- jd : Julian day in which the image was acquired (used to identify crop stage)
- long : longitude
- lat : latitude
- Crop : type of crop, cross referenced with USDA Cropland Data Layer.
- Stage : crop's stage of growth, cross referenced with Center for Sustainability and the Global Environment

In [11]:
data.isnull().sum()

UniqueID       0
Country        0
XAEZ           0
Image          0
Month          0
            ... 
X2355       6988
X2365       6988
X2375       6988
X2385       6988
X2395       6988
Length: 209, dtype: int64

Note that reflectance has been consistently removed in five bands of the spectrum (X2355, X2365, X2375, X2385, X2395). We can transform the nulls to 0 in this notebook to visualize the available spectrum in its integrity. When modeling, we will drop these columns.

In [12]:
data = data.fillna(value=0)

As *UniqueID* is different for each row, we will set that column as an index.

In [13]:
data.set_index('UniqueID', inplace=True)

Because all data come from the USA, we can drop this column in the analysis.

In [14]:
data.drop(columns = ['Country'], inplace= True)

## Grouping signatures of crops

We will create hyperspectral dataframes with different subsets (either grouping per crop or grouping per crop per stage).  For convenience, let's make a copy of the hyperspectral dataset.

In [16]:
data2 = data.copy(deep=True)

In [17]:
data2.head()

,XAEZ,Image,Month,Year,jd,long,lat,Crop,Stage,X427,...,X2305,X2315,X2325,X2335,X2345,X2355,X2365,X2375,X2385,X2395
UniqueID,,,,,,,,,,,,,,,,,,,,,
1466,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.663419,38.534516,corn,Critical,0.0,...,7.380592,7.327904,6.881876,6.616288,6.346634,0.0,0.0,0.0,0.0,0.0
1467,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.671589,38.504744,corn,Critical,0.0,...,8.090403,7.894839,7.263033,6.536649,5.663291,0.0,0.0,0.0,0.0,0.0
1469,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.597588,38.614056,corn,Critical,0.0,...,5.116303,4.833205,4.817732,4.652525,3.790456,0.0,0.0,0.0,0.0,0.0
1470,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.687293,38.571702,corn,Critical,0.0,...,6.291307,6.922908,6.943876,6.461839,5.696439,0.0,0.0,0.0,0.0,0.0
1476,7,EO1H0440332012234110KD_SGS_01,8,2012,234,-121.625189,38.572225,rice,Early_Mid,0.0,...,6.007986,4.820400,3.084297,1.322807,1.087548,0.0,0.0,0.0,0.0,0.0


In [18]:
data2.drop(columns = ['XAEZ', 'Image', 'Month', 'Year',
                    'jd', 'long', 'lat'], inplace= True)

In [19]:
data2.head()

,Crop,Stage,X427,X437,X447,X457,X468,X478,X488,X498,...,X2305,X2315,X2325,X2335,X2345,X2355,X2365,X2375,X2385,X2395
UniqueID,,,,,,,,,,,,,,,,,,,,,
1466,corn,Critical,0.0,17.724022,16.105696,15.024523,14.715921,14.441503,14.166527,13.830132,...,7.380592,7.327904,6.881876,6.616288,6.346634,0.0,0.0,0.0,0.0,0.0
1467,corn,Critical,0.0,17.850463,15.999081,15.013466,14.782901,14.501259,14.257706,13.966769,...,8.090403,7.894839,7.263033,6.536649,5.663291,0.0,0.0,0.0,0.0,0.0
1469,corn,Critical,0.0,17.566144,15.844705,14.903485,14.906514,14.673835,14.297178,13.872969,...,5.116303,4.833205,4.817732,4.652525,3.790456,0.0,0.0,0.0,0.0,0.0
1470,corn,Critical,0.0,17.509762,15.825789,14.863198,14.759544,14.411682,14.147443,13.909676,...,6.291307,6.922908,6.943876,6.461839,5.696439,0.0,0.0,0.0,0.0,0.0
1476,rice,Early_Mid,0.0,17.540088,15.498984,14.596975,14.237083,13.915319,13.491013,12.943444,...,6.007986,4.820400,3.084297,1.322807,1.087548,0.0,0.0,0.0,0.0,0.0


## Visualizing reflectance average values per crop

Before diving into Panel, let’s create a function that smooths one of our time series and identifies outliers. Then, we’ll plot the result using hvPlot:

In [20]:
c_unique = data2.Crop.unique()

In [21]:
c_unique

array(['corn', 'rice', 'cotton', 'soybean', 'winter_wheat'], dtype=object)

In [22]:
crop1 = c_unique[0]
crop2 = c_unique[1]
crop3 = c_unique[2]
crop4 = c_unique[3]
crop5 = c_unique[4]

In [25]:
variable = 'Crop'

In [26]:
data2

,Crop,Stage,X427,X437,X447,X457,X468,X478,X488,X498,...,X2305,X2315,X2325,X2335,X2345,X2355,X2365,X2375,X2385,X2395
UniqueID,,,,,,,,,,,,,,,,,,,,,
1466,corn,Critical,0.0,17.724022,16.105696,15.024523,14.715921,14.441503,14.166527,13.830132,...,7.380592,7.327904,6.881876,6.616288,6.346634,0.0,0.0,0.0,0.0,0.0
1467,corn,Critical,0.0,17.850463,15.999081,15.013466,14.782901,14.501259,14.257706,13.966769,...,8.090403,7.894839,7.263033,6.536649,5.663291,0.0,0.0,0.0,0.0,0.0
1469,corn,Critical,0.0,17.566144,15.844705,14.903485,14.906514,14.673835,14.297178,13.872969,...,5.116303,4.833205,4.817732,4.652525,3.790456,0.0,0.0,0.0,0.0,0.0
1470,corn,Critical,0.0,17.509762,15.825789,14.863198,14.759544,14.411682,14.147443,13.909676,...,6.291307,6.922908,6.943876,6.461839,5.696439,0.0,0.0,0.0,0.0,0.0
1476,rice,Early_Mid,0.0,17.540088,15.498984,14.596975,14.237083,13.915319,13.491013,12.943444,...,6.007986,4.820400,3.084297,1.322807,1.087548,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13424,soybean,Critical,0.0,19.002399,17.121772,16.297446,15.874480,15.508477,14.901770,14.425840,...,6.763665,6.813507,7.034937,7.364454,7.585006,0.0,0.0,0.0,0.0,0.0
13427,soybean,Critical,0.0,19.727713,17.425258,16.291641,15.913486,15.603432,15.006249,14.453180,...,6.690982,6.483347,5.838236,5.723123,5.664319,0.0,0.0,0.0,0.0,0.0
13431,soybean,Critical,0.0,19.853683,17.228156,16.094961,15.927941,15.599947,15.136182,14.439636,...,7.664325,6.842533,6.197422,5.654009,5.611239,0.0,0.0,0.0,0.0,0.0


In [27]:
def transform_data(variable, crop1, crop2, crop3, crop4, crop5):
    """Calculates the rolling average and identifies outliers"""
    a = data2.index[data2[variable] == crop1].tolist()
    b = data2.index[data2[variable] == crop2].tolist()
    c = data2.index[data2[variable] == crop3].tolist()
    d = data2.index[data2[variable] == crop4].tolist()
    e = data2.index[data2[variable] == crop5].tolist()
    data3 = data2[data2.index.isin(a)]
    data4 = data3.drop(columns=['Crop', 'Stage'])
    avg1 = data4.mean(axis=0)
    data3 = data2[data2.index.isin(b)]
    data4 = data3.drop(columns=['Crop', 'Stage'])
    avg2 = data4.mean(axis=0)
    data3 = data2[data2.index.isin(c)]
    data4 = data3.drop(columns=['Crop', 'Stage'])
    avg3 = data4.mean(axis=0)
    data3 = data2[data2.index.isin(d)]
    data4 = data3.drop(columns=['Crop', 'Stage'])
    avg4 = data4.mean(axis=0)
    data3 = data2[data2.index.isin(e)]
    data4 = data3.drop(columns=['Crop', 'Stage'])
    avg5 = data4.mean(axis=0)
    return avg1, avg2, avg3, avg4, avg5

In [28]:
avg1, avg2, avg3, avg4, avg5 = transform_data(variable, crop1, crop2, crop3, crop4, crop5)

In [29]:
def get_plot(variable, crop1, crop2, crop3, crop4, crop5):
    """Plots the average for each crop"""
    avg1, avg2, avg3, avg4, avg5 = transform_data(variable, crop1, crop2, crop3, crop4, crop5)
    return avg1.hvplot(
        height=300, 
        legend=True, 
        color=COLOR1, 
        line_width=3, 
        label= variable + ' = ' + str(crop1)).opts(title="Average reflectance") * avg2.hvplot(color=COLOR2, legend=True, label= str(variable + ' = ' + str(crop2)
        )
        )  * avg3.hvplot(color=COLOR3, legend=True, label= str(variable + ' = ' + str(crop3)
        )
                        ) * avg4.hvplot(color=COLOR4, legend=True, label= str(variable + ' = ' + str(crop4)
        )
                        ) * avg5.hvplot(color=COLOR5, legend=True, label= str(variable + ' = ' + str(crop5)
        ))

Now, we can call our get_plot function with specific parameters to obtain a plot with a single set of parameters:

In [30]:
get_plot(variable, crop1,crop2, crop3, crop4, crop5)

:Overlay
   .Curve.Crop_equals_corn         :Curve   [index]   (0)
   .Curve.Crop_equals_rice         :Curve   [index]   (0)
   .Curve.Crop_equals_cotton       :Curve   [index]   (0)
   .Curve.Crop_equals_soybean      :Curve   [index]   (0)
   .Curve.Crop_equals_winter_wheat :Curve   [index]   (0)

Great! Now, it may be helpful to look for other vegetation studies and compare these signatures with spectral responses from same crops in other regions. 

## Visualizing average reflectance values per crop per stage

In [60]:
data2 = data.copy(deep=True)

In [61]:
data2.drop(columns = ['XAEZ', 'Image', 'Month', 'Year',
                    'jd', 'long', 'lat'], inplace= True)

In [62]:
data2.Stage.unique()

array(['Critical', 'Early_Mid', 'Late', 'Mature_Senesc', 'Harvest',
       'Emerge_VEarly'], dtype=object)

In [63]:
s_unique = data2.Stage.unique()

In [64]:
stage1 = s_unique[0]
stage2 = s_unique[1]
stage3 = s_unique[2]
stage4 = s_unique[3]
stage5 = s_unique[4]
stage6 = s_unique[5]

In [65]:
cultivo = 'rice'

In [66]:
crop = data2.index[data2['Crop'] == cultivo].tolist()

In [67]:
data3 = data2[data2.index.isin(crop)]

In [68]:
data3

,Crop,Stage,X427,X437,X447,X457,X468,X478,X488,X498,...,X2305,X2315,X2325,X2335,X2345,X2355,X2365,X2375,X2385,X2395
UniqueID,,,,,,,,,,,,,,,,,,,,,
1476,rice,Early_Mid,0.0,17.540088,15.498984,14.596975,14.237083,13.915319,13.491013,12.943444,...,6.007986,4.820400,3.084297,1.322807,1.087548,0.0,0.0,0.0,0.0,0.0
1477,rice,Early_Mid,0.0,16.648189,15.031340,14.110798,13.906287,13.656374,13.224093,12.693501,...,3.664176,3.505168,3.140695,2.344013,1.717203,0.0,0.0,0.0,0.0,0.0
1478,rice,Early_Mid,0.0,17.699469,15.695622,14.780258,14.845887,14.698671,14.296352,13.902163,...,5.339100,5.784331,5.645195,5.039616,4.173784,0.0,0.0,0.0,0.0,0.0
1479,rice,Early_Mid,0.0,18.828728,17.082747,16.035258,15.752055,15.508538,15.234029,15.070395,...,7.319944,6.602842,6.216662,6.655033,7.260858,0.0,0.0,0.0,0.0,0.0
1480,rice,Early_Mid,0.0,18.093873,16.324612,15.369231,15.028049,14.726458,14.402038,14.010762,...,6.692794,6.788359,6.341197,5.248109,4.318360,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9415,rice,Late,0.0,21.838206,19.499573,18.266393,18.089558,17.563515,17.014603,16.402195,...,7.638025,7.492359,6.687474,5.744078,5.406034,0.0,0.0,0.0,0.0,0.0
9416,rice,Late,0.0,22.304704,20.321629,19.040001,18.996970,18.604142,18.195390,17.855209,...,10.171074,9.962615,8.579084,7.413975,7.173328,0.0,0.0,0.0,0.0,0.0
9417,rice,Late,0.0,21.943569,19.490475,18.011418,17.734097,17.153336,16.713732,16.127963,...,6.285087,6.340614,5.636300,6.021593,6.592214,0.0,0.0,0.0,0.0,0.0


In [69]:
def ntransform_data(cultivo, stage1, stage2, stage3, stage4, stage5, stage6):
    """Calculates the rolling average response per crop per stage"""
    crop = data2.index[data2['Crop'] == cultivo].tolist()
    data3 = data2[data2.index.isin(crop)]
    stage =  data3.index[data3['Stage'] == stage1].tolist()
    data4 = data3[data3.index.isin(stage)]
    data5 = data4.drop(columns=['Crop', 'Stage'])
    avg1 = data5.mean(axis=0)
    stage =  data3.index[data3['Stage'] == stage2].tolist()
    data4 = data3[data3.index.isin(stage)]
    data5 = data4.drop(columns=['Crop', 'Stage'])
    avg2 = data5.mean(axis=0)
    stage =  data3.index[data3['Stage'] == stage3].tolist()
    data4 = data3[data3.index.isin(stage)]
    data5 = data4.drop(columns=['Crop', 'Stage'])
    avg3 = data5.mean(axis=0)
    stage =  data3.index[data3['Stage'] == stage4].tolist()
    data4 = data3[data3.index.isin(stage)]
    data5 = data4.drop(columns=['Crop', 'Stage'])
    avg4 = data5.mean(axis=0)
    stage =  data3.index[data3['Stage'] == stage5].tolist()
    data4 = data3[data3.index.isin(stage)]
    data5 = data4.drop(columns=['Crop', 'Stage'])
    avg5 = data5.mean(axis=0)
    stage =  data3.index[data3['Stage'] == stage6].tolist()
    data4 = data3[data3.index.isin(stage)]
    data5 = data4.drop(columns=['Crop', 'Stage'])
    avg6 = data5.mean(axis=0)
    return avg1, avg2, avg3, avg4, avg5, avg6

In [70]:
def nget_plot(variable, stage1, stage2, stage3, stage4, stage5, stage6):
    """Plots the average for each stage for a given crop"""
    avg1, avg2, avg3, avg4, avg5, avg6 = ntransform_data(variable, stage1, stage2, stage3, stage4, stage5, stage6)
    return avg1.hvplot(
        height=300, 
        legend=True, 
        color=COLOR1, 
        line_width=3, 
        label= variable + ' = ' + str(stage1)).opts(title="Average reflectance") * avg2.hvplot(color=COLOR2, legend=True, label= str(variable + ' = ' + str(stage2)
        )
        )  * avg3.hvplot(color=COLOR3, legend=True, label= str(variable + ' = ' + str(stage3)
        )
                        ) * avg4.hvplot(color=COLOR4, legend=True, label= str(variable + ' = ' + str(stage4)
        )
                        ) * avg5.hvplot(color=COLOR5, legend=True, label= str(variable + ' = ' + str(stage5)
        )
                        ) * avg6.hvplot(color=COLOR6, legend=True, label= str(variable + ' = ' + str(stage6)
        ))

In [71]:
cultivo = 'winter_wheat'
nget_plot(cultivo, stage1,stage2, stage3, stage4, stage5, stage6)

:Overlay
   .Curve.Winter_wheat_equals_Critical      :Curve   [index]   (0)
   .Curve.Winter_wheat_equals_Early_Mid     :Curve   [index]   (0)
   .Curve.Winter_wheat_equals_Late          :Curve   [index]   (0)
   .Curve.Winter_wheat_equals_Mature_Senesc :Curve   [index]   (0)
   .Curve.Winter_wheat_equals_Harvest       :Curve   [index]   (0)
   .Curve.Winter_wheat_equals_Emerge_VEarly :Curve   [index]   (0)

In [72]:
variable = 'cotton'
nget_plot(variable, stage1,stage2, stage3, stage4, stage5, stage6)

:Overlay
   .Curve.Cotton_equals_Critical      :Curve   [index]   (0)
   .Curve.Cotton_equals_Early_Mid     :Curve   [index]   (0)
   .Curve.Cotton_equals_Late          :Curve   [index]   (0)
   .Curve.Cotton_equals_Mature_Senesc :Curve   [index]   (0)
   .Curve.Cotton_equals_Harvest       :Curve   [index]   (0)
   .Curve.Cotton_equals_Emerge_VEarly :Curve   [index]   (0)

In [73]:
variable = 'rice'
nget_plot(variable, stage1,stage2, stage3, stage4, stage5, stage6)

:Overlay
   .Curve.Rice_equals_Critical      :Curve   [index]   (0)
   .Curve.Rice_equals_Early_Mid     :Curve   [index]   (0)
   .Curve.Rice_equals_Late          :Curve   [index]   (0)
   .Curve.Rice_equals_Mature_Senesc :Curve   [index]   (0)
   .Curve.Rice_equals_Harvest       :Curve   [index]   (0)
   .Curve.Rice_equals_Emerge_VEarly :Curve   [index]   (0)

In [74]:
# You can explore  here different combinations of crop & stages
#
#

Again, it is worth to compare these signatures with other obtained in similar studies. In addition, it can be interesting to have a look at the [Aneece & Thenkabail (2018) paper](https://www.mdpi.com/2072-4292/10/12/2027)

![](./GHISA/signatures3.png)

## Further improvement of functions

The functions written here for transforming and averaging hyperspectral signatures of different crops and stages seems to be very rudimentary. Thus, in case you can write better functions, please let me know to improve this notebook.

## Further tasks

For those of you interested in hyperspectral crop classification using the GHISA dataset, see [this Kaggle notebook](https://www.kaggle.com/code/billbasener/ghisaconus-hyperspectral-crop-classification).

## References

Aneece, I. &  Thenkabail, PS.. 2018. Accuracies achieved in classifying five leading world crop types and their growth stages using optimal Earth Observing-1 Hyperion hyperspectral narrowbands on Google Earth Engine. Remote Sensing. DOI: 10.3390/rs10122027

Cavender-Bares, J., Gamon J.A., and Townsend, P.A. 2020. Remote Sensing of Plant Biodiversity. Springer.